In [1]:
from email.mime import base
from re import A
import torch
import torch.nn as nn
import torch.nn.functional as F 
import torchvision
import numpy as np
import matplotlib.pyplot as plt
from cone_dataset import build_dataset, CustomDataset
from ConeSegmentation.CGNetPy1 import CGNet, CGNetEnd
import time
import pynvml

In [2]:
def infernece_test(model, model_end, dataset, num_tests=100, device=torch.device('cpu')):

    times1 = np.zeros([num_tests])
    times2 = np.zeros([num_tests])
    model.eval()
    model_end.eval()
    model.to(device)
    model_end.to(device)

    for idx, (image, mask) in enumerate(dataset):
        then = time.time()
        out = model(image.unsqueeze(0).to(device))
        dt1 = time.time() - then
        out = model_end(out)
        dt2 = time.time() - then

        times1[idx] = dt1
        times2[idx] = dt2

        if idx == num_tests - 1:
            break
    
    return np.mean(times1), np.mean(times2)

In [4]:
dataset = CustomDataset()
img, mask = dataset[0]

model = CGNet().cpu()
model_end = CGNetEnd().cpu()

state_dict = torch.load('ConeSegmentation/weights/CGNetWeights_conv_4')
model.load_state_dict(state_dict)

<All keys matched successfully>

In [5]:
for _ in range(10):
    time1, time2 = infernece_test(model, model_end, dataset, num_tests=400, device=torch.device('cpu'))
    print(time1, time2)

0.12084499001502991 0.13004222750663758
0.11644684672355651 0.12524299085140228
0.11701663672924041 0.12583650290966034
0.11675618290901184 0.1255663400888443
0.11673943400382995 0.12552351176738738
0.1163004207611084 0.12511396765708924
0.11600773394107819 0.12480874598026276
0.11618974924087525 0.12494357764720916
0.11651590466499329 0.12531755089759827
0.11711815953254699 0.12595398008823394


In [7]:
def make_binary(array):
    base_array = np.zeros_like(array)
    base_array[array > 0.5] = 1

    return base_array

In [8]:
def accuracy_test(model, model_end, dataset, num_tests=100, device=torch.device('cpu')):

    accuacy = np.zeros([num_tests])
    model.eval()
    model_end.eval()
    model.to(device)
    model_end.to(device)

    for idx in range(num_tests):
# for idx, (image, mask) in enumerate(dataset):
        image, mask = dataset[np.random.randint(0, 1000)]
        out = model(image.unsqueeze(0).to(device))
        out = model_end(out)

        out = make_binary(out[0].cpu().detach().numpy())
        mask = make_binary(mask.cpu().detach().numpy())

        intersection = np.logical_and(out[0], mask)
        union = np.logical_or(out[0], mask)

        # plt.imshow(np.transpose(mask, (1, 2, 0)))
        # plt.savefig('mask.png')
        # plt.imshow(np.transpose(out, (1, 2, 0)))
        # plt.savefig('image.png')
        # plt.imshow(np.transpose(intersection, (1, 2, 0)))
        # plt.savefig('intersection.png')
        # plt.imshow(np.transpose(union, (1, 2, 0)))
        # plt.savefig('union.png')

        iou = np.sum(intersection).item() / np.sum(union).item() if np.sum(union).item() > 0 else 1
        accuacy[idx] = iou

        if idx == num_tests - 1:
            break
    
    return np.mean(accuacy)

In [9]:
for _ in range(10):
    accuracy = accuracy_test(model, model_end, dataset, num_tests=400, device=torch.device('cpu'))
    print(accuracy)

0.646419080588062
0.6488117909495784
0.6453196813041435
0.6380291363786937
0.6460422861234028
0.6583027168311023
0.6407602686642639
0.6526778736011455
0.6450010703659075
0.6397798648199519


In [10]:
def power_test(model, model_end, dataset, num_tests=100, device=torch.device('cuda')):

    torch.cuda.set_device(0)

    pynvml.nvmlInit()
    gpu = pynvml.nvmlDeviceGetHandleByIndex(0)
    name = pynvml.nvmlDeviceGetName(gpu)
#     device = name.decode("utf-8")

    powers = np.zeros([num_tests])
    model.eval()
    model_end.eval()
    model.to(device)
    model_end.to(device)

    for idx, (image, mask) in enumerate(dataset):
        out = model(image.unsqueeze(0).to(device))
        out = model_end(out)
        power_usage = pynvml.nvmlDeviceGetPowerUsage(gpu) / 1000

        powers[idx] = power_usage

        if idx == num_tests - 1:
            break
    
    return np.mean(powers)

In [11]:
for _ in range(10):
    power = power_test(model, model_end, dataset, num_tests=10, device=torch.device('cpu'))
    print(power)

6.2158
6.08
6.051
6.0659
6.1238
6.153500000000001
6.0706
6.0416
6.0803
6.182


In [12]:
for _ in range(10):
    time1, time2 = infernece_test(model, model_end, dataset, num_tests=400, device=torch.device('cuda'))
    print(time1, time2)

0.013975104689598084 0.014048531651496887
0.014037632942199707 0.014111154675483704
0.014129211902618408 0.014203370213508607
0.014052704572677613 0.014126431941986085
0.013924887180328369 0.013998509645462036
0.01401820957660675 0.01409163773059845
0.01407098412513733 0.014144925475120545
0.014060906171798705 0.014134681224822998
0.014029946923255921 0.014103278517723083
0.013960661888122559 0.014033951163291931


In [13]:
for _ in range(10):
    accuracy = accuracy_test(model, model_end, dataset, num_tests=400, device=torch.device('cuda'))
    print(accuracy)

0.641515020201476
0.6619012616596368
0.6337050426290716
0.6460170640375341
0.6364644236777329
0.6459393529295963
0.6510115294649269
0.6506078215904724
0.6481219363808836
0.6433354182425517


In [14]:
for _ in range(10):
    power = power_test(model, model_end, dataset, num_tests=10, device=torch.device('cuda'))
    print(power)

49.249900000000004
49.0556
48.6913
49.352
49.273
48.8284
49.1006
49.3017
49.2608
49.03490000000001
